In [22]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm

In [25]:
pDir = './result/CMUnet_R2C/BU_ST_UD_QA/BU_ST_UD_QA'

##### (1) 구성 dataset별 iou. dice score 비교

In [26]:
df = pd.read_csv(f'{pDir}/result.csv', index_col=0)
df.head()

,ID,task,iou,dice,SE,PE,F1,SP,ACC
0,BUSI_malignant_167,BUSI,0.922366,0.959615,0.936819,0.983549,0.959615,0.996511,0.985641
1,BUSI_malignant_41,BUSI,0.792595,0.884299,0.829412,0.946966,0.884299,0.996221,0.983673
2,BUSI_benign_378,BUSI,0.878510,0.935326,0.958751,0.913019,0.935326,0.997572,0.996567
3,BUSI_malignant_148,BUSI,0.763547,0.865922,0.769868,0.989362,0.865921,0.999923,0.997803
4,BUSI_benign_30,BUSI,0.928922,0.963151,0.992147,0.935802,0.963151,0.999197,0.999115


In [27]:
df.groupby(by='task')['iou', 'dice'].agg(['mean', 'count'])

/tmp/ipykernel_438573/1947295277.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby(by='task')['iou', 'dice'].agg(['mean', 'count'])


iou            dice      
            mean count      mean count
task                                  
BUSI    0.739171   128  0.816291   128
QAMEBI  0.785502    46  0.857348    46
STU     0.860057     8  0.924005     8
UDIAT   0.750422    32  0.827784    32

---
##### (2) Visualization

In [18]:
def draw_ctrs(img, mask, text, color = (255, 255, 255)):
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    ctrs, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for ctr in ctrs:
        img = cv2.drawContours(img, [ctr], -1, color, 2)
        cv2.putText(img, text, ((min(ctr[:,0,0]) + max(ctr[:,0,0]))//2-25, max(ctr[:,0,1])+35), 4, 1, color, 2, cv2.LINE_AA)
    return img

os.makedirs(f'{pDir}/visualization',exist_ok=True)

In [ ]:

for id in tqdm(os.listdir(f'{pDir}/predictions')):
    task = id.split('_')[0]
    name = id.replace(f'{task}_', '')
    
    dice = (round(df[df['ID'] == id.replace('.png', '')].iloc[0]['dice'], 3))
    img         = cv2.imread(f"inputs/{task}/imagesTr/{name}")
    mask        = cv2.imread(f"inputs/{task}/labelsTr/{name}")
    pred        = cv2.imread(f"{pDir}/predictions/{id}")
    img, mask, pred   = cv2.resize(img, (256, 256)), cv2.resize(mask, (256, 256)), cv2.resize(pred, (256, 256))
    
    origin  = img.copy()
    img_with_gt = draw_ctrs(img, mask, 'GT')
    
    img2 = origin.copy()
    img_with_pred = draw_ctrs(img2, pred, 'Pred', color=(0,0,255))
    
    # draw
    plt.cla()
    f, axes = plt.subplots(1, 3)
    axes[0].axis(False)
    axes[0].imshow(origin)
    axes[0].set_title('Image')#(id.replace('.png', ''))
    axes[1].axis(False)
    axes[1].imshow(img_with_gt)
    axes[1].set_title('GT')
    axes[2].axis(False)
    axes[2].imshow(img_with_pred)
    axes[2].set_title(f'Prediction({dice})')
    plt.suptitle(id.replace('.png', ''))
    f.tight_layout()
    plt.savefig(f'{pDir}/visualization/{id}', bbox_inches='tight', pad_inches=0, dpi=400)
